C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64>python -m pip install google-cloud-bigquery

In [1]:
import pyodbc 
from datetime import datetime, date, timedelta
from google.cloud import bigquery
from google.oauth2 import service_account

In [4]:
src_conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=Localhost;'
                      'Database=CollectionStrategy;'
                      'Trusted_Connection=yes;')

credentials = service_account.Credentials.from_service_account_file('/data/docker/ProsperAirflow1_10_x/dags/keys/datalake-poc-205120.json')
project_id = 'datalake-poc-205120'
client = bigquery.Client(credentials=credentials,project=project_id)
# Prepares a reference to the dataset
dataset_ref = client.dataset('DW')
table_ref = dataset_ref.table('treatment1')
table = client.get_table(table_ref)  # API call

In [5]:
rowbatch = 2000

#load variables from destination
startdate = date.today()-timedelta(days=1) # 2019-07-11 done (next date has no rows)
enddate = date.today()-timedelta(days=1)

print(startdate)
print(enddate)

2019-07-14
2019-07-14


In [7]:
cursor = src_conn.cursor()

while enddate <= startdate:

    rowcount = 0
    rowinsert = []
    
    cursor.execute("SELECT LOAN_ID, PROCESS_DATE FROM dbo.TREATMENT \
                    WHERE PROCESS_DATE = '{}'".format(str(startdate)))
    #check for rows
    if cursor.rowcount == 0:
        rowcount = -1 # used to escape final batch insert and move to next date
    
    for row in cursor:
        rowinsert += [(row[0],row[1])]
        rowcount += 1
        
        #insert batch of rows
        if rowcount == rowbatch:
            errors = client.insert_rows(table, rowinsert)  # API request
            assert errors == []
            rowcount = 0
            rowinsert = []
    
    #insert remaining rows when smaller than batch
    if rowcount != rowbatch and rowcount != -1:
        errors = client.insert_rows(table, rowinsert)  # API request
        assert errors == []
    
    print(startdate)
    startdate = startdate-timedelta(days=1)
         
cursor.close()

2019-07-14
